# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [92]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [78]:
vacay="output_data/cities.csv"
v_df = pd.read_csv(vacay)
v_df

,City,Lat,Lng,Humidity,Cloudiness,WindSpeed,Temps
0,atuona,-9.80,-139.03,75,3,21.54,78.94
1,vaini,-21.20,-175.20,74,20,12.75,80.60
2,punta arenas,-53.15,-70.92,81,90,9.17,48.20
3,ribeira grande,38.52,-28.70,68,42,21.23,61.48
4,kruisfontein,-34.00,24.73,83,100,7.00,68.00
...,...,...,...,...,...,...,...
586,masallatah,32.62,14.00,63,33,9.51,66.06
587,cap-aux-meules,47.38,-61.86,40,0,14.99,33.80
588,muzhi,65.37,64.67,86,39,17.60,35.58
589,cayenne,4.93,-52.33,100,90,3.36,77.00


In [79]:
locations = v_df[["Lat", "Lng"]]
H_rating = v_df["Humidity"].astype(float)

In [80]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights= H_rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [81]:
perfect_spot_df = v_df.loc[(v_df["WindSpeed"]<10)&(v_df["Cloudiness"]==0)&(v_df["WindSpeed"]<10)&(v_df["Temps"].between(75,85))]
perfect_spot_df = perfect_spot_df.reset_index(drop =True, inplace =False)
perfect_spot_df

,City,Lat,Lng,Humidity,Cloudiness,WindSpeed,Temps
0,bowen,-20.02,148.23,65,0,2.24,78.80
1,nioaque,-21.14,-55.83,38,0,3.58,76.68
2,formosa,-26.18,-58.18,31,0,8.05,78.80
3,veraval,20.90,70.37,59,0,6.76,78.51
4,gorontalo,0.54,123.06,81,0,1.50,79.41
5,berbera,10.44,45.01,61,0,0.94,77.74
6,kaduqli,11.02,29.72,36,0,7.49,80.26
7,srivardhan,18.03,73.02,62,0,5.93,76.21
8,karratha,-20.74,116.85,29,0,6.58,82.49
9,lalmohan,22.34,90.74,79,0,8.77,75.15


In [105]:
params = {    
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in perfect_spot_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    Url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    try:
        hotel_address = requests.get(Url,params=params)
        hotel_address= hotel_address.json()
        perfect_spot_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
        perfect_spot_df.loc[index, "Country"] = hotel_address["results"][0]["plus_code"]["compound_code"]
    except IndexError:
        print("No hotel nearby")
    pprint(hotel_address)


{'html_attributions': [],
 'next_page_token': 'AcnaJv1eH9OamvZYkShssLzn6l0vC1FVqcCFIYABun0KgKjmpIHe4G6iiFRvx3RwtMHoQlxvGmoSoVY-Q5-d7SaauwlD7-TzCU4Qk3MnNQgP7TK5zgBJekdFB83mr3kUPk_gWudRTAPJKo-ll0xVKuWJSg9Hgd1Pbwu4Po1ppoGiAUuQwnlwLK2F_F3jC7q0FHh95Lb48thjEsUEov3rN9DYifT-7rTdhug_TOPCCIH9DZ1Dg0nMQQXhnmXJW0fyX1RDsNkK5IpNbILPoTJB6VTeI0VPIsbu0zU6Eqo4oOHNjAW0aTxaPFeJj5AlsL2C8-Atg2UTnYCcjjBeSJTfZXpzKqx4bre5DxKbS1y9ZPq413dOUizFibivvXBKciBePpVvxoj4k9Z6Njld0Xve-8OImDUuDiMkgw1s93SgXuzAG-3EMUtFRk-DCi9eSkkK',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -19.9805939, 'lng': 148.2601833},
                           'viewport': {'northeast': {'lat': -19.97902392010727,
                                                      'lng': 148.2613397298927},
                                        'southwest': {'lat': -19.98172357989272,
                                                      'lng': 148.2586400701072}}},
              'icon': 'https://maps.gstatic.com

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -21.1601643, 'lng': -55.8525925},
                           'viewport': {'northeast': {'lat': -21.15875767010728,
                                                      'lng': -55.85123517010727},
                                        'southwest': {'lat': -21.16145732989272,
                                                      'lng': -55.85393482989272}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Nioaque Novo Hotel',
              'photos': [{'height': 1080,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/104334921743250306585">silvia '
                                                'aparecida</a>'],
                          'photo_reference': 'CmRaAAAA8VSALYAAnApG-LtdP1WihtvewQ29P9r9E

{'html_attributions': [],
 'next_page_token': 'AcnaJv3JrR6q1PsBZS0xr3e0ZKOIfCUSbDD5yvH7-6dT8F1HI9zm5_HCCK5KaKGe1sGI9G-5CJQqw5yDFUk6slTxWjveheFg5Pf8VqJQqmw3J2_AgUFG-Q77-C4BWlyGf4pAZVY3mYb4vHymq0YLXNrJmKOtSJgplPfG1ieF65LVkQSXpsy1C1GF4VxmaYBVIQiCJbJ1It6RHqwIy9QkfiH9X3KmZY6LxT1Qb-gLYSeFM0ROkQl-7JCuNOaa8e5U_OESqGfUHZIfq26XjTZFqVKyRs4lXcuHduxV5kNajcVMO5q7CzPsJuTgfApEwfAVn2YByJTy0SzHZOVoM_3cXQlFIhKcq_N7S0tJbQ_L0a1_ysx8uedFIV16L8tANQpb4_xJnWvuQ5-BBSvggwkkpZ5R9Emo9jB_gD9kXLBnoYRrTo0w5iS4HdGVZVHwgWM7',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 20.901515, 'lng': 70.4059475},
                           'viewport': {'northeast': {'lat': 20.90284402989272,
                                                      'lng': 70.4069913298927},
                                        'southwest': {'lat': 20.90014437010727,
                                                      'lng': 70.40429167010727}}},
              'icon': 'https://maps.gstatic.com/mapfi

{'html_attributions': [],
 'next_page_token': 'AcnaJv0cVkmSgjkg_O8z-i2RgJak9_Wm11mglV0jWF0PiB4pkz38rd_9EBqPKWg4G4pzVaG-6chXX4yJeA4aVno4PDVH7w6jNqNtvpyTDLbDaWLJUAPTO99XnPz0fAYH7nAzMd5-1TvxHs0az6t4UdKqXCX_escmuiJrB5T4PT5lkj0DUJkICFEufKTLjniCMaP-OzInS1Jqd8yPl0Q5H4CPhjDXNGg_cf-sHYYcLimJPKqvMmdWgcLz1Us8Aj8-t5mMc2WtzPYTvuSi85JeWzL0Yq5XmMUwlxz-trH2YY0UVGx2CnX_AMw8_J_cfoidLg-NTqKpeTs68nYTR0gz0m-Q4dNdxbStH42ULe9AzzI0WmgfKXe7TUBVIfJUjGJlyE50jjiCjhfCVPU3Sw3WLMqwLXlA-qWeHrDB9bbvLvU3_G0lSPs3pjgyrdtYhysG',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 0.5378187999999999,
                                        'lng': 123.0626416},
                           'viewport': {'northeast': {'lat': 0.5392270298927221,
                                                      'lng': 123.0640022798927},
                                        'southwest': {'lat': 0.5365273701072777,
                                                      'lng': 123.0613026201073}}},


{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 10.440097,
                                        'lng': 45.00898369999999},
                           'viewport': {'northeast': {'lat': 10.44146772989272,
                                                      'lng': 45.01042462989272},
                                        'southwest': {'lat': 10.43876807010728,
                                                      'lng': 45.00772497010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Damal Hotel',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 721,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/100954568078943762055">A '
                                                'Google User</a>'],
       

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -20.7244552, 'lng': 116.8660446},
                           'viewport': {'northeast': {'lat': -20.72317257010727,
                                                      'lng': 116.8673825798927},
                                        'southwest': {'lat': -20.72587222989272,
                                                      'lng': 116.8646829201073}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
              'name': 'Latitude20 The Dunes Apartments',
              'photos': [{'height': 380,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/108464832422298321706">Karratha '
                                                'Dunes Apartments</a>'],
                          'photo_reference': 'CmRaAAAAuUt9B

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 22.340285,
                                        'lng': 90.73475959999999},
                           'viewport': {'northeast': {'lat': 22.34165567989272,
                                                      'lng': 90.73607532989273},
                                        'southwest': {'lat': 22.33895602010728,
                                                      'lng': 90.73337567010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Arafat Hotel',
              'photos': [{'height': 3456,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/107315122539437774136">Zahidul '
                                                'Islam</a>'],
                          'photo_reference': 'CmRaAAAA30

In [104]:
hotel_address["results"][0]["plus_code"]["compound_code"]

'MJR6+G4 Port Hedland, Western Australia, Australia'

In [106]:
perfect_spot_df

,City,Lat,Lng,Humidity,Cloudiness,WindSpeed,Temps,Hotel Name,Country
0,bowen,-20.02,148.23,65,0,2.24,78.80,Horseshoe Bay Resort,"2796+Q3 Bowen, Queensland, Australia"
1,nioaque,-21.14,-55.83,38,0,3.58,76.68,Nioaque Novo Hotel,"R4QW+WX São Miguel, Nioaque - State of Mato Gr..."
2,formosa,-26.18,-58.18,31,0,8.05,78.80,Casagrande Apart Hotel,"RRGJ+97 Formosa, Formosa Province, Argentina"
3,veraval,20.90,70.37,59,0,6.76,78.51,THE SOMNATH GATEWAY,"WC24+J9 Somnath, Gujarat, India"
4,gorontalo,0.54,123.06,81,0,1.50,79.41,Maqna Hotel by Prasanthi,"G3Q7+43 Heledulaa Selatan, Kota Gorontalo, Gor..."
5,berbera,10.44,45.01,61,0,0.94,77.74,Damal Hotel,"C2R5+2H Berbera, Somalia"
6,kaduqli,11.02,29.72,36,0,7.49,80.26,فندق الله جابو,"2P59+FH Kadugli, Sudan"
7,srivardhan,18.03,73.02,62,0,5.93,76.21,Tranquil Beach Resort,"2229+7H Harihareshwar, Maharashtra, India"
8,karratha,-20.74,116.85,29,0,6.58,82.49,Latitude20 The Dunes Apartments,"7VG8+6C Bulgarra, Western Australia, Australia"
9,lalmohan,22.34,90.74,79,0,8.77,75.15,Arafat Hotel,"8PRM+4W Lalmohan, Bangladesh"


In [124]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perfect_spot_df.iterrows()]
locations = perfect_spot_df[["Lat", "Lng"]]

# Add marker layer ontop of heat map
locations = perfect_spot_df[["Lat", "Lng"]]
H_rating = perfect_spot_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights= H_rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

# Display figure

markers = gmaps.marker_layer(perfect_spot_df[["Lat", "Lng"]])
fig.add_layer(markers)
plt.savefig("output_data/Hotel_map.png")
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>